In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import rrule
import ahmedsabri
from ahmedsabri import *
%matplotlib inline

In [ ]:
start_date = datetime(2022, 1, 1) # year,month,day
end_date = datetime(2023, 2, 25) # year, month, day

In [ ]:
def prepare(data):
    df=data.copy()
    df=df.transpose()
    df.columns=df.iloc[0:3].fillna('').astype(str).apply(' '.join).str.strip()
    df=df.iloc[3:]
    #df.columns=df.columns.str.replace(" ","_")
    #df.columns=df.columns.str.lower()
    #df.columns=df.columns.str.replace("-","_")
    df.reset_index(drop=True, inplace=True)
    #df.dropna(how="any", thresh=df.shape[0]*0.99,inplace=True)
    #df.dropna(axis="columns", how="any", thresh=df.shape[0]*0.99, inplace=True)
    #df=df.replace('No Data',np.nan)
    #df=df.replace('No Lab Data',np.nan)
    #df=df.replace('NaT',np.nan)
    #df=df.replace('-',np.nan)
    df=df.replace('Nil',0)
    return df

In [ ]:
def lab_assurance(df):
    for i in range (1,df.shape[0]):
        if type(df.iloc[i,0])!=str:
            df.iloc[i,0]=df.iloc[i-1,0]
    #df.dropna(axis=0,how='any',thresh=6,inplace=True)
    #df.reset_index(inplace=True,drop=True)

In [ ]:
# making dictionary which keys is month_year and values are the dataframes
df_macro_dict={}
df_lab_dict={}
df_calcs_dict={}
for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
    os.chdir(os.path.join(r'\\192.168.21.16\Area 02A sharing\Spread Sheets',str(dt.year)))
    datetime_object = datetime.strptime(str(dt.month), "%m")
    month = datetime_object.strftime("%b")
    year=str(dt.year)[-2:]
    sheet_name='Unit 25 SS' +' '+ month.capitalize() +' '+ year+' '+'.xlsm'
    #df='df'+'_'+str(dt.month)+'_'+str(dt.year)[-2:]
    df_macro_dict[month+'_'+year]=pd.read_excel(sheet_name ,header = None,sheet_name='Macro',usecols="a:aj",nrows=121)
    df_lab_dict[month+'_'+year]=pd.read_excel(sheet_name,header = None,sheet_name="Lab data input",usecols="a:aj",nrows=320)
    df_calcs_dict[month+'_'+year]=pd.read_excel(sheet_name,header = None,sheet_name="Calculations",usecols="a:ag",nrows=32)

## prepare macro and concacenate it

In [ ]:
for key in df_macro_dict.keys():
    df_macro_dict[key]=prepare(df_macro_dict[key])
    if df_macro_dict[key].columns.duplicated().sum()>0:
        df_macro_dict[key] = df_macro_dict[key].loc[:,~df_macro_dict[key].columns.duplicated()].copy()
    df_macro_dict[key]['Description Units Tag No.']=pd.to_datetime(df_macro_dict[key]['Description Units Tag No.'],errors='coerce')
    df_macro_dict[key].dropna(subset=['Description Units Tag No.'],inplace=True)
    df_macro_dict[key].set_index('Description Units Tag No.',inplace=True)

In [ ]:
df_macro=pd.concat(df_macro_dict.values(),axis=0)

## preparing Lab and concacenate it

In [ ]:
for key in df_lab_dict.keys():
    try:
        lab_assurance(df_lab_dict[key])
    except:
        continue

In [ ]:
for key in df_lab_dict.keys():
    df_lab_dict[key]=prepare(df_lab_dict[key])
    if df_lab_dict[key].columns.duplicated().sum()>0:
        df_lab_dict[key] = df_lab_dict[key].loc[:,~df_lab_dict[key].columns.duplicated()].copy()
    df_lab_dict[key]['Sample Test Method']=pd.to_datetime( df_lab_dict[key]['Sample Test Method'],errors='coerce')
    df_lab_dict[key].dropna(subset=['Sample Test Method'],inplace=True)
    df_lab_dict[key].set_index('Sample Test Method',inplace=True)    

In [ ]:
#https://www.geeksforgeeks.org/merge-two-dataframes-with-same-column-names/
df_lab=pd.concat(df_lab_dict.values(),axis=0)

# MERGING and saving to CSV

In [ ]:
df_macro.shape

In [ ]:
df_lab.shape

In [ ]:
df_macro = df_macro[~df_macro.index.duplicated(keep="first")]

In [ ]:
df_lab = df_lab[~df_lab.index.duplicated(keep="first")]

In [ ]:
df=pd.concat([df_macro, df_lab], axis=1)

In [ ]:
os.chdir(r'\\192.168.21.16\Area 02A sharing\Spread Sheets\spread sheets database by python')

In [ ]:
df.to_csv(f'U_25 spreadsheet{str(start_date).split()[0]}_to_{str(end_date).split()[0]}.csv')

## Converting all data to numbers

In [ ]:
# to convert object only ( not dates or numbers)
df[df.dtypes[df.dtypes=='object'].index].columns

In [ ]:
df.shape

In [ ]:
df= df.loc[:,~df.columns.duplicated()]

In [ ]:
df.shape

In [ ]:
for column in df[df.dtypes[df.dtypes=='object'].index].columns:
    df[column]=pd.to_numeric(df[column],errors='coerce')

In [ ]:
plotting(df)

In [ ]:
#df.to_csv('orange_data.csv')

In [ ]:
os.getcwd()